In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import copy,time
#import pandas as pd
import scipy.sparse as sp
from scipy.sparse.linalg import eigs
import h5py
#import networkx as nx
import straw
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [3]:
def Laplacian(adjacency):
        """计算 L=D^-0.5 * (A+I) * D^-0.5"""
        #adjacency += sp.eye(adjacency.shape[0])    # 增加自连接
        degree = np.array(adjacency.sum(1))
        d_hat = sp.diags(np.power(degree, -0.5).flatten())
        return d_hat.dot(adjacency).dot(d_hat).tocoo()
        #return sp.diags(degree.flatten())-adjacency.tocoo()

In [1]:
chrom_length = {'chr1': 249250621,
 'chr2': 243199373,
 'chr3': 198022430,
 'chr4': 191154276,
 'chr5': 180915260,
 'chr6': 171115067,
 'chr7': 159138663,
 'chr8': 146364022,
 'chr9': 141213431,
 'chr10': 135534747,
 'chr11': 135006516,
 'chr12': 133851895,
 'chr13': 115169878,
 'chr14': 107349540,
 'chr15': 102531392,
 'chr16': 90354753,
 'chr17': 81195210,
 'chr18': 78077248,
 'chr19': 59128983,
 'chr20': 63025520,
 'chr21': 48129895,
 'chr22': 51304566,
 'chrX':  156000000}

In [ ]:
chrom_coor = {}
chrom_all_length = 0
for key,value in chrom_length.items():
    chrom_coor[key] = chrom_all_length
    chrom_all_length = chrom_all_length+value
    
bin_size = 100000
matrix_size = int(np.floor(chrom_all_length/bin_size))
chrom_sign = np.zeros(matrix_size)
k = 0
for i in chrom_length.keys():
    k = k+1
    chrom_sign[np.floor(chrom_coor[i]/bin_size).astype(int):]=k
    
cgisign = chrom_sign.copy()
cgipos = np.loadtxt("/home/chrom_region/hg19_CGI_chrall.bed",dtype=str,usecols=[0,1,2])
k = 0
for i in chrom_length.keys():
    k = k+1
    cgipos_chr = cgipos[cgipos[:,0]==i][:,[1,2]].astype(int).mean(1)
    cgicounts = np.histogram(cgipos_chr,range = (0,np.sum(chrom_sign==k)*bin_size),bins = np.sum(chrom_sign==k))[0]
    cgisign[chrom_sign==k] = cgicounts

In [ ]:
samples = np.loadtxt("/home/hic/celllist.txt",dtype = str)

In [21]:
chrall_HiC_allsample = []
for sample in samples:
    chrall_HiC = sp.coo_matrix(([0],([0],[0])),shape = (matrix_size,matrix_size),dtype = "float32")
    for chrA in range(23):
        for chrB in np.arange(chrA,23):
            HiC = straw.straw('NONE','/home/hic/'+sample,chrom[chrA],chrom[chrB],'BP',bin_size)
            #print(max(HiC[1]))
            row = np.floor(np.array(HiC[0])/bin_size)+np.floor(chrom_coor['chr'+chrom[chrA]]/bin_size)
            col = np.floor(np.array(HiC[1])/bin_size)+np.floor(chrom_coor['chr'+chrom[chrB]]/bin_size)
            HiC = sp.coo_matrix((np.array(HiC[2]),(row,col)),shape = (matrix_size,matrix_size),dtype = "float32")
            chrall_HiC = chrall_HiC+HiC
    chrall_HiC_allsample.append(chrall_HiC)

In [ ]:
comp_index_allsample = []
for i in range(samples.size):
    HiC = chrall_HiC_allsample[i].copy()
    HiC = HiC+HiC.transpose()
    q_99 = np.quantile(HiC.data,0.99)
    HiC.data[HiC.data>q_99] = q_99
    s = np.array(HiC.sum(1)).reshape(-1)
    HiC = HiC[s>0]
    HiC = HiC[:,s>0]
    Lap = Laplacian(HiC)

    vals, vecs = eigs(Lap, k=100)
    vecs = np.real_if_close(vecs)
    vals = np.real_if_close(vals)
    o = np.argsort(1-vals)
    vals = 1-vals[o]
    vecs = vecs[:,o]

    lda = LinearDiscriminantAnalysis().fit_transform(vecs,(cgisign>0)[s>0])
    lda = 1/(1+np.exp(-lda))-0.5
    comp_index = np.repeat(np.nan,s.shape[0])
    comp_index[s>0] = lda.reshape(-1)

    comp_index_allsample.append(comp_index)
    #break
comp_index_allsample = np.array(comp_index_allsample)